In [1]:
import os 
import pandas as pd
import numpy as np
import re, json
from tqdm import tqdm

df = pd.read_excel(os.path.join('/Users/huni/Dropbox/내 Mac (MacBook-Pro.local)/Downloads', '(DB)221107.xlsx'))
df.shape

/Users/huni/.pyenv/versions/3.8.2/envs/{huni}/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


(10716, 23)

In [2]:
df.columns

Index(['#', 'Unnamed: 0.1', '제목', '서식', '서식2', '서식3', '작가', 'ISBN', 'CuriPick',
       '수상/추천', 'CuriLevel', '대상연령', '도서정보', '주제', '시리즈', '북레벨(AR)', '렉사일',
       '상품평개수', '도서번호', 'Unnamed: 0', '출판사', '대분류', '소분류'],
      dtype='object')

In [3]:
df.drop(['#', 'Unnamed: 0.1', '서식2', '서식3'], axis = 1, inplace=True)

df.columns

Index(['제목', '서식', '작가', 'ISBN', 'CuriPick', '수상/추천', 'CuriLevel', '대상연령',
       '도서정보', '주제', '시리즈', '북레벨(AR)', '렉사일', '상품평개수', '도서번호', 'Unnamed: 0',
       '출판사', '대분류', '소분류'],
      dtype='object')

In [4]:
df.drop(['Unnamed: 0'], axis = 1, inplace=True)

df.columns

Index(['제목', '서식', '작가', 'ISBN', 'CuriPick', '수상/추천', 'CuriLevel', '대상연령',
       '도서정보', '주제', '시리즈', '북레벨(AR)', '렉사일', '상품평개수', '도서번호', '출판사', '대분류',
       '소분류'],
      dtype='object')

In [5]:
df.shape

(10716, 18)

In [6]:
df_curi = pd.read_excel(os.path.join('/Users/huni/Dropbox/내 Mac (MacBook-Pro.local)/Downloads', 'CuriPick List_20221110.xlsx'))
df_curi.shape

(385, 5)

In [7]:
df_curi.columns

Index(['Title', 'Author', 'Synopsis', 'Barcode', ' Collection/Wishlist'], dtype='object')

In [8]:
count_matched = 0
count_matched_over = 0 

for idx, row in tqdm(df_curi.iterrows(), desc = ">>> Proceeding"):
    df_match = df[df['제목']==row['Title']]
    if df_match.shape[0] > 0:
        count_matched += 1 
        if df_match.shape[0] > 1: #!# 아마 저자가 다를 것임
            count_matched_over += 1 
            print('> compare: ', df_match['작가'].values)

>>> Proceeding: 385it [00:00, 602.08it/s]


In [9]:
count_matched

4

In [10]:
import copy
df_n = copy.deepcopy(df)

df_n['제목'] = df['제목'].apply(lambda x: str(x).lower())
df_n['제목'].values[:10]

array(['1999-06-29 00:00:00', '...and now miguel ',
       '[세이펜book] barbie readers 바비 리더스 레벨 1 : 리더스북 12권 + 오디오 cd 1장 + 액티비티북 1권',
       '[특가] percy jackson and the olympians #1 : the lightning thief ',
       "[comic rockets] step 2: mr croc's silly sock",
       '[eq의 천재들] little miss리틀미스:my complete collection',
       "[weird planet] #01 : dube, where's my spaceship? ",
       '#02 the flying beaver brothers and the fishy business ',
       '#02. who was anne frank? ', '#03 : shoo, fly guy! '], dtype=object)

- 맨 뒤에 공백 제거

In [11]:
def delSpace(inpText):
    if inpText[-1] == ' ':
        return inpText[:-1]
    else:
        return inpText

df_n['제목'] = df_n['제목'].apply(lambda x: delSpace(x))
df_n['제목'].values[8:20]

array(['#02. who was anne frank?', '#03 : shoo, fly guy!',
       '#03 the flying beaver brothers and the mud-slinging moles',
       '#04 : there was an old lady who swallowed fly guy',
       '#04 the flying beaver brothers: birds vs. bunnies',
       '#05 : fly high, fly guy!', '#06 : hooray for fly guy!',
       '#06 the flying beaver brother and the crazy critter race',
       '#07 : i spy fly guy', '#08 : fly guy meets fly girl',
       '#09 : buzz boy and fly guy',
       "#1. escape from mr. lemoncello's library"], dtype=object)

In [12]:
df_curi['Title'] = df_curi['Title'].apply(lambda x: str(x).lower())
df_curi['Title'].values[:10]

array(['#015: cherry the cake fairy', '#016: melodie the music fairy',
       '#021: jasmine the present fairy', '100 facts ancient rome',
       'a single shard', 'a tooth story',
       'ada byron lovelace and the thinking machine',
       'abigail the breeze fairy', 'al capone does my shirts',
       'al capone shines my shoes'], dtype=object)

In [13]:
count_matched = 0
count_matched_over = 0

for idx, row in tqdm(df_curi.iterrows(), desc = ">>> Proceeding"):
    df_match = df_n[df_n['제목']==row['Title']]
    if df_match.shape[0] > 0:
        count_matched += 1

        if df_match.shape[0] > 1: #!# 아마 저자가 다를 것임
            count_matched_over += 1 
            print('> compare: ', df_match['제목'].values[0], '.....', df_match['작가'].values)

>>> Proceeding: 282it [00:00, 667.36it/s]

> compare:  holes ..... ['Louis Sachar' 'Louis Sachar , Vladimir Radunsky , Bagram Ibatoulline']
> compare:  number the stars ..... ['Lois Lowry' 'Lois Lowry']
> compare:  penguin ..... ['Claire Masurel , True Kelley' 'Joan Holub , Julie Durrell'
 'DyAnne DiSalvo-Ryan' 'Gail Herman , Norman Gorbaty'
 'DyAnne DiSalvo-Ryan' 'Judith Moffatt' 'Emily Neye' 'Kate Spohn'
 'Eric Carle' 'Harriet Ziefert' 'Joan Holub , Lynne Avril Cravath'
 'Janet Morgan Stoeke' 'Joan Holub , Hiroe Nakata'
 'Nadine Bernard Westcott , Barbara Maitland' 'Joanna Cole , Ron Broda'
 'Joan Holub , Rich Davis' 'Jill Abramson , Deborah Melmon' 'Pam Pollack'
 'Laura Marchesani , Zenaides A. , Jr. Medina , Peter Jarvis'
 'R. A. Herman' 'DyAnne DiSalvo-Ryan' 'Harriet Ziefert'
 'Martin Silverman , Amy Aitken' 'Joseph Slate , Ashley wolff'
 'Susan Hood , Kristina Stephenson' 'Tracey West , Cindy Revell'
 'Rosemary Wells , Andrew Grey' 'Rosemary Wells , Andrew Grey'
 'Jonathan Fenske' 'Anna Dewdney' 'Tomie DePaola'
 'Simms Ta

>>> Proceeding: 385it [00:00, 649.41it/s]

> compare:  the giver ..... ['Lois Lowry' 'Lois Lowry']
> compare:  the miraculous journey of edward tulane ..... ['Kate Dicamillo' 'Kate Dicamillo , Bagram Ibatoulline']
> compare:  when i grow up ..... ['Mercer Mayer' 'Al Yankovic']


In [14]:
count_matched

84

In [15]:
for idx, row in tqdm(df_curi.iterrows(), desc = ">>> Proceeding"):
    df_match = df_n[df_n['제목']==row['Title']]
    for idx in df_match.index:
        df_n.loc[idx, 'CuriPick'] = 1


df_n[df_n['CuriPick'] == 1].shape

>>> Proceeding: 385it [00:00, 726.54it/s]


(129, 18)

In [16]:
df_n['CuriPick'].values

array([nan, nan, nan, ..., nan, nan, nan])

In [17]:
df_n['CuriPick'] = df_n['CuriPick'].fillna(0)
df_n[df_n['CuriPick']== 0].shape

(10587, 18)

In [18]:
df['CuriPick'] = df_n['CuriPick']

In [22]:
df[df['CuriPick'] == 1].shape

(128, 18)

In [20]:
should_be_delete = {
    'the giver': 'Lois Lowry'
}

to_Drop = []
for key, val in should_be_delete.items():
    df_slice = df_n[df_n['제목'] == key]
    print('>>> ', df_slice.index)
    print('>>> ', df_slice['작가'].values)
    for author in df_slice['작가'].values:

        if  author == val:
            print('> ', df_slice['서식'].values)
            # print('> ', df_slice.index)
            # to_Drop.append()

>>>  Int64Index([8714, 8715], dtype='int64')
>>>  ['Lois Lowry' 'Lois Lowry']
>  [nan '책 + MP3 오디오시디) 기억 전달자']
>  [nan '책 + MP3 오디오시디) 기억 전달자']


In [21]:
# 지울 때는 df를 지워야함, lower case 안 먹혀있어서, 이게 제대로된 것
print(df.shape)
df.drop([8714], axis=0, inplace=True)
print(df.shape)

(10716, 18)
(10715, 18)


In [ ]:
df.to_excel('(DB)221110.xlsx')